In [1]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 12.6 MB/s eta 0:00:0000:0100:01


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import faiss

from tqdm import tqdm
from scipy.spatial import distance

pd.set_option('max_colwidth', 320)
pd.set_option('display.width', 600)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/samokattechworkshop/base.csv
/kaggle/input/samokattechworkshop/train.csv
/kaggle/input/samokattechworkshop/test.csv
/kaggle/input/samokattechworkshop/baseline.ipynb
/kaggle/input/samokattechworkshop/answer_sample.csv


In [3]:
%%time
base_df = pd.read_csv('/kaggle/input/samokattechworkshop/base.csv')
train_df = pd.read_csv('/kaggle/input/samokattechworkshop/train.csv')
test_df = pd.read_csv('/kaggle/input/samokattechworkshop/test.csv')
answer_sample = pd.read_csv('/kaggle/input/samokattechworkshop/answer_sample.csv')

CPU times: user 36.9 s, sys: 2.76 s, total: 39.7 s
Wall time: 1min 1s


In [4]:
only_train = train_df.drop_duplicates(subset='Target')['Target']
only_base = base_df.query('Id in @only_train')
train_df.drop_duplicates(subset='Target')['Target'].nunique()

delta_df = np.abs(only_base.set_index('Id') - train_df.drop_duplicates(subset='Target').set_index('Target').drop('Id', axis=1))

threshold = 2000000 #2 000 000 # 
delta_df_drop_trash = delta_df.loc[:, delta_df.sum() > threshold]
trash_col = delta_df_drop_trash.columns

base_df = base_df.drop(trash_col, axis=1)
train_df = train_df.drop(trash_col, axis=1)
test_df = test_df.drop(trash_col, axis=1)

In [5]:
base_df.head()

,Id,0,1,2,3,4,5,7,8,9,...,60,61,62,63,64,66,67,68,69,71
0,0-base,-115.083890,11.152912,-64.426760,-118.88089,216.482440,-104.698060,44.348083,120.915344,181.449700,...,-64.518950,-138.777630,-42.808693,38.800827,-151.76218,63.666340,-4.703861,92.933610,115.269190,-60.830353
1,1-base,-34.562202,13.332763,-69.787610,-166.53348,57.680607,-86.098370,-35.637436,119.718636,195.234190,...,11.392853,-148.571660,-117.767525,41.100000,-157.82940,68.202110,24.346846,179.937930,116.834000,-59.524610
2,2-base,-54.233746,6.379371,-29.210136,-133.41383,150.895830,-99.435326,62.381706,128.951450,164.381470,...,-101.581375,-134.658220,-76.397800,46.011803,-207.14442,65.566180,66.325680,81.073490,116.594154,-32.527206
3,3-base,-87.520130,4.037884,-87.803030,-185.06763,76.369540,-58.985165,-33.611237,122.031910,136.233580,...,-89.588715,-149.326690,-70.647940,-6.358921,-147.20105,66.202890,-20.566910,137.206940,117.474100,-72.915490
4,4-base,-72.743850,6.522049,43.671265,-140.60803,5.820023,-112.074080,45.182500,122.167180,112.119064,...,-100.565520,-125.317330,-57.199104,56.642403,-159.35184,66.766320,-2.505783,65.315285,135.051590,0.319401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2918134,4744755-base,-125.086300,4.735448,38.284935,-125.27671,209.788360,-47.870010,-71.077340,125.534355,29.341850,...,-62.274580,-135.932040,-70.364395,55.428260,-191.53702,65.515260,108.963715,-52.795380,117.254510,-77.768555
2918135,4744759-base,-108.598390,5.454173,-22.191975,-158.71666,113.606110,-88.843430,137.317690,129.056690,164.754240,...,-90.002280,-123.761055,-70.360190,18.439003,-90.65145,68.691666,1.636499,-24.388298,93.530540,-90.800400
2918136,4744762-base,-97.026730,-2.143885,-71.274090,-144.68066,149.464810,-21.572212,-108.151955,124.810135,118.760700,...,-147.723750,-118.983600,-55.961933,62.923470,-169.11832,68.592970,43.205433,69.029830,129.536650,4.718300
2918137,4744763-base,-86.838430,8.911945,-34.837738,-106.46268,42.236572,-96.194230,9.845185,126.037560,59.223100,...,-50.803364,-135.374420,-86.867020,-6.959065,-174.07591,68.622246,0.707927,94.451866,124.843000,-55.789314


In [6]:
# def round_to_nearest_5(x):
#     return round(x/5) * 5

In [7]:
base = base_df.drop(['Id'], axis=1).copy().round(1)
train = train_df.drop(['Id', 'Target'], axis=1).copy().round(1)
test = test_df.drop(['Id'], axis=1).copy().round(1)

# base_vectors = base_data.iloc[:, :-1].values
# train_vectors = train_data.iloc[:, :-1].values
# test_vectors = test_data.iloc[:, :-1].values

In [8]:
# base = base.applymap(round_to_nearest_5)
# train = train.applymap(round_to_nearest_5)
# test = test.applymap(round_to_nearest_5)

In [9]:
# Подготовка данных Извлечение векторов из датасета
data_vectors = np.ascontiguousarray(base.values.astype('float32'))
train_vectors = np.ascontiguousarray(train.values.astype('float32'))
test_vectors = np.ascontiguousarray(test.values.astype('float32'))

In [10]:
# data_vectors = data_vectors / np.linalg.norm(data_vectors, axis=1)[:, None]
# train_vectors = train_vectors / np.linalg.norm(train_vectors, axis=1)[:, None]
# test_vectors = test_vectors / np.linalg.norm(test_vectors, axis=1)[:, None]

In [11]:
index = faiss.IndexFlatIP(base.shape[1])  # L2 метрика для расстояний
faiss.normalize_L2(data_vectors)
res = faiss.StandardGpuResources()  # Доступ к ресурсам GPU
gpu_index = faiss.index_cpu_to_gpu(res, 0, index)  # Индекс на GPU
gpu_index.add(data_vectors)

In [12]:
answer = pd.DataFrame(index=test_df['Id'])
answer['Predicted'] = None

In [13]:
num = 0
k = 10 #10 for predict 30 for some work
for vector in tqdm(test_vectors[:]):
    distances, indices = gpu_index.search(vector.reshape(1, -1), k)
    answer['Predicted'].iloc[num] = ' '.join(base_df.iloc[indices.flatten()]['Id'].values)
    num +=1

100%|██████████| 100000/100000 [23:18<00:00, 71.51it/s]


In [14]:
answer.to_csv('answer_30.csv')